# importing libraries

In [10]:
import nltk
import tensorflow
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [11]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [12]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [13]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

59 documents
15 classes ['Food', 'Mask', 'Music', 'Sanitizer', 'cases', 'corona', 'goodbye', 'greeting', 'love', 'medicines', 'options', 'precautions', 'symptoms', 'thanks', 'vaccine']
100 unique lemmatized words ["'s", ',', 'a', 'against', 'am', 'anyone', 'are', 'available', 'avoid', 'avoidable', 'avoidance', 'awesome', 'be', 'best', 'book', 'bot', 'bye', 'can', 'case', 'chatting', 'check', 'choice', 'corona', 'could', 'covid19', 'day', 'do', 'eating', 'effective', 'favourite', 'food', 'for', 'genre', 'get', 'good', 'goodbye', 'guideline', 'hand', 'having', 'healthy', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'if', 'immunity', 'is', 'know', 'later', 'love', 'make', 'mask', 'me', 'meal', 'medicine', 'more', 'music', 'my', 'neccesary', 'neccessary', 'next', 'nice', 'number', 'of', 'offered', 'patient', 'precaution', 'prevent', 'provide', 'safe', 'sanitizer', 'santizer', 'see', 'should', 'support', 'symptom', 'tell', 'thank', 'thanks', 'that', 'the', 'there'

# creating training data

In [14]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


# training the model

In [16]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=5, verbose=1)
model.save('our_model.h5', hist)

print("model created")

Epoch 1/500
12/12 [==============================] - 0s 831us/step - loss: 2.7712 - accuracy: 0.0339
Epoch 2/500
12/12 [==============================] - 0s 1ms/step - loss: 2.6604 - accuracy: 0.0847
Epoch 3/500
12/12 [==============================] - 0s 1ms/step - loss: 2.6179 - accuracy: 0.1356
Epoch 4/500
12/12 [==============================] - 0s 1ms/step - loss: 2.5442 - accuracy: 0.1695
Epoch 5/500
12/12 [==============================] - 0s 1ms/step - loss: 2.5590 - accuracy: 0.1186
Epoch 6/500
12/12 [==============================] - 0s 915us/step - loss: 2.4442 - accuracy: 0.1695
Epoch 7/500
12/12 [==============================] - 0s 1ms/step - loss: 2.3412 - accuracy: 0.2542
Epoch 8/500
12/12 [==============================] - 0s 1ms/step - loss: 2.3159 - accuracy: 0.3390
Epoch 9/500
12/12 [==============================] - 0s 1ms/step - loss: 2.1445 - accuracy: 0.4068
Epoch 10/500
12/12 [==============================] - 0s 1ms/step - loss: 1.9884 - accuracy: 0.3898
Epoch

12/12 [==============================] - 0s 831us/step - loss: 0.1003 - accuracy: 0.9831
Epoch 83/500
12/12 [==============================] - 0s 665us/step - loss: 0.0606 - accuracy: 1.0000
Epoch 84/500
12/12 [==============================] - 0s 748us/step - loss: 0.1907 - accuracy: 0.9322
Epoch 85/500
12/12 [==============================] - 0s 997us/step - loss: 0.0497 - accuracy: 1.0000
Epoch 86/500
12/12 [==============================] - 0s 997us/step - loss: 0.1737 - accuracy: 0.9322
Epoch 87/500
12/12 [==============================] - 0s 1ms/step - loss: 0.1111 - accuracy: 0.9831
Epoch 88/500
12/12 [==============================] - 0s 997us/step - loss: 0.1044 - accuracy: 0.9831
Epoch 89/500
12/12 [==============================] - 0s 997us/step - loss: 0.1496 - accuracy: 0.9492
Epoch 90/500
12/12 [==============================] - 0s 997us/step - loss: 0.1624 - accuracy: 0.9661
Epoch 91/500
12/12 [==============================] - 0s 831us/step - loss: 0.1129 - accuracy: 0.

12/12 [==============================] - 0s 997us/step - loss: 0.0214 - accuracy: 1.0000
Epoch 163/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0345 - accuracy: 1.0000
Epoch 164/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0411 - accuracy: 0.9831
Epoch 165/500
12/12 [==============================] - 0s 997us/step - loss: 0.0511 - accuracy: 0.9831
Epoch 166/500
12/12 [==============================] - 0s 748us/step - loss: 0.0775 - accuracy: 0.9831
Epoch 167/500
12/12 [==============================] - 0s 665us/step - loss: 0.0218 - accuracy: 1.0000
Epoch 168/500
12/12 [==============================] - 0s 831us/step - loss: 0.0304 - accuracy: 1.0000
Epoch 169/500
12/12 [==============================] - 0s 831us/step - loss: 0.0545 - accuracy: 0.9831
Epoch 170/500
12/12 [==============================] - 0s 914us/step - loss: 0.0117 - accuracy: 1.0000
Epoch 171/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0337 - accurac

12/12 [==============================] - 0s 1ms/step - loss: 0.0194 - accuracy: 0.9831
Epoch 243/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0170 - accuracy: 1.0000
Epoch 244/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0236 - accuracy: 1.0000
Epoch 245/500
12/12 [==============================] - 0s 997us/step - loss: 0.0104 - accuracy: 1.0000
Epoch 246/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0123 - accuracy: 1.0000
Epoch 247/500
12/12 [==============================] - 0s 915us/step - loss: 0.0050 - accuracy: 1.0000
Epoch 248/500
12/12 [==============================] - 0s 831us/step - loss: 0.0215 - accuracy: 0.9831
Epoch 249/500
12/12 [==============================] - 0s 748us/step - loss: 0.0380 - accuracy: 0.9831
Epoch 250/500
12/12 [==============================] - 0s 748us/step - loss: 0.0080 - accuracy: 1.0000
Epoch 251/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0252 - accuracy: 1

12/12 [==============================] - 0s 997us/step - loss: 0.0244 - accuracy: 1.0000
Epoch 323/500
12/12 [==============================] - 0s 997us/step - loss: 0.0738 - accuracy: 0.9661
Epoch 324/500
12/12 [==============================] - 0s 997us/step - loss: 0.0434 - accuracy: 0.9661
Epoch 325/500
12/12 [==============================] - 0s 748us/step - loss: 0.0100 - accuracy: 1.0000
Epoch 326/500
12/12 [==============================] - 0s 665us/step - loss: 0.0662 - accuracy: 0.9831
Epoch 327/500
12/12 [==============================] - 0s 914us/step - loss: 0.0052 - accuracy: 1.0000
Epoch 328/500
12/12 [==============================] - 0s 997us/step - loss: 0.0029 - accuracy: 1.0000
Epoch 329/500
12/12 [==============================] - 0s 997us/step - loss: 0.0152 - accuracy: 1.0000
Epoch 330/500
12/12 [==============================] - 0s 914us/step - loss: 0.0382 - accuracy: 0.9831
Epoch 331/500
12/12 [==============================] - 0s 914us/step - loss: 0.0071 - a

12/12 [==============================] - 0s 1ms/step - loss: 0.0095 - accuracy: 1.0000
Epoch 402/500
12/12 [==============================] - 0s 997us/step - loss: 0.0134 - accuracy: 1.0000
Epoch 403/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 404/500
12/12 [==============================] - 0s 997us/step - loss: 0.0538 - accuracy: 0.9831
Epoch 405/500
12/12 [==============================] - 0s 997us/step - loss: 0.0081 - accuracy: 1.0000
Epoch 406/500
12/12 [==============================] - 0s 748us/step - loss: 0.0774 - accuracy: 0.9831
Epoch 407/500
12/12 [==============================] - 0s 665us/step - loss: 0.0336 - accuracy: 0.9831
Epoch 408/500
12/12 [==============================] - 0s 914us/step - loss: 0.0136 - accuracy: 1.0000
Epoch 409/500
12/12 [==============================] - 0s 997us/step - loss: 0.0295 - accuracy: 0.9831
Epoch 410/500
12/12 [==============================] - ETA: 0s - loss: 0.0546 - accuracy: 1

Epoch 481/500
12/12 [==============================] - 0s 914us/step - loss: 0.0076 - accuracy: 1.0000
Epoch 482/500
12/12 [==============================] - 0s 997us/step - loss: 0.0324 - accuracy: 0.9831
Epoch 483/500
12/12 [==============================] - 0s 997us/step - loss: 0.0433 - accuracy: 0.9831
Epoch 484/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0052 - accuracy: 1.0000
Epoch 485/500
12/12 [==============================] - 0s 998us/step - loss: 0.0161 - accuracy: 1.0000
Epoch 486/500
12/12 [==============================] - 0s 831us/step - loss: 0.0092 - accuracy: 1.0000
Epoch 487/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0105 - accuracy: 1.0000
Epoch 488/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 489/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 490/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0055